In [ ]:
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import xavier_uniform_
from torch.nn.utils import weight_norm
import numpy as np
import torchvision.utils as vutils
from torchvision.utils import save_image
import random
import os
import shutil
import pdb
from logger import Logger
from PIL import Image

In [ ]:
# Initialization
num_channels = 3
num_classes = 1
num_epochs = 300
image_size = 32
batch_size = 64
epsilon = 1e-8 # used to avoid NAN loss
logger = Logger('./logs')

# Initialize parameters
lr = 1e-5
b1 = 0.5 # adam: decay of first order momentum of gradient
b2 = 0.999 # adam: decay of first order momentum of gradient

model_path ='./baseline.tar'
image_dir = 'baseline_images'

os.makedirs(image_dir, exist_ok=True)

In [ ]:
# Create Dataset
class TCGADataset(Dataset):
    def __init__(self, image_size, split):
        self.split = split
        self.tcga_dataset = self._create_dataset(image_size, split)
        self.patches, self.labels = self.tcga_dataset
        self.transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize((.5, .5, .5), (.5, .5, .5))
        ])
        
    def _create_dataset(self, image_size, split):
        data_dir = '../dataset/patch_data'
        if self.split == 'train':
            data_dir = os.path.join(data_dir, 'train')
        else:
            data_dir = os.path.join(data_dir, 'dev')
        
        all_files = os.listdir(data_dir)
        images = []
        labels = []
        
        # Iterate over all files
        for file in all_files:
            if '.npz' not in file:
                continue
            file_path = os.path.join(data_dir, file)
            data = np.load(file_path)
            X = data['arr_0']
            y = data['arr_1']
            images.append(X)
            labels.append(y)                
            
        images = np.concatenate(images)
        labels = np.concatenate(labels) 
        
        # Print data statistics
        print("Total number of patches : ",labels.shape[0])
        c = 0
        nc = 0
        for l in labels:
            if l:
                c+=1
            else:
                nc+=1
        print("Cancerous patches : ",c )
        print("Non cancerous patches : ",nc )
            
        return images, labels
    

    def __getitem__(self, idx):
        data, label = self.patches[idx], self.labels[idx]
        return self.transform(Image.fromarray(data)), label

    def __len__(self):
        return len(self.labels)

In [ ]:
# Get dataloaders
def get_loader(image_size, batch_size):
    num_workers = 2
    tcga_train = TCGADataset(image_size=image_size, split='train')
#     tcga_test = TCGADataset(image_size=image_size, split='test')

    train_loader = DataLoader(
        dataset=tcga_train,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers
    )

#     test_loader = DataLoader(
#         dataset=tcga_test,
#         batch_size=batch_size,
#         shuffle=True
#         #num_workers=num_workers
#     )

    return train_loader#, test_loader

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.zero_()
        
        
def initializer(m):
    # Run xavier on all weights and zero all biases
    if hasattr(m, 'weight'):
        if m.weight.ndimension() > 1:
            xavier_uniform_(m.weight.data)

    if hasattr(m, 'bias') and m.bias is not None:
        m.bias.data.zero_() 

In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
          
        dropout_rate = 0.5
        filter1 = 96
        filter2 = 192
        
        # Conv operations
        # CNNBlock 1
        self.wn_conv1 = nn.Sequential(
            weight_norm(nn.Conv2d(in_channels=num_channels, out_channels=filter1, kernel_size=3, stride=1, padding=1), name='weight'),
            nn.LeakyReLU(0.2),
            weight_norm(nn.Conv2d(in_channels=filter1, out_channels=filter1, kernel_size=3, stride=1, padding=1), name='weight'),
            nn.LeakyReLU(0.2),
            weight_norm(nn.Conv2d(in_channels=filter1, out_channels=filter1, kernel_size=3, stride=2, padding=1), name='weight'),
            nn.LeakyReLU(0.2),
            nn.Dropout2d(dropout_rate)
        )
        
        # CNNBlock 2
        self.wn_conv2 = nn.Sequential(
            weight_norm(nn.Conv2d(in_channels=filter1, out_channels=filter2, kernel_size=3, stride=1, padding=1), name='weight'),
            nn.LeakyReLU(0.2),
            weight_norm(nn.Conv2d(in_channels=filter2, out_channels=filter2, kernel_size=3, stride=1, padding=1), name='weight'),
            nn.LeakyReLU(0.2),
            weight_norm(nn.Conv2d(in_channels=filter2, out_channels=filter2, kernel_size=3, stride=2, padding=1), name='weight'),
            nn.LeakyReLU(0.2),
            nn.Dropout2d(dropout_rate)
        )
        
        # CNNBlock 3
        self.wn_conv3 = nn.Sequential(
            weight_norm(nn.Conv2d(in_channels=filter2, out_channels=filter2, kernel_size=3, stride=1, padding=0), name='weight'),
            nn.LeakyReLU(0.2),
            weight_norm(nn.Conv2d(in_channels=filter2, out_channels=filter2, kernel_size=1, stride=1, padding=0), name='weight'),
            nn.LeakyReLU(0.2),
            weight_norm(nn.Conv2d(in_channels=filter2, out_channels=filter2, kernel_size=1, stride=1, padding=0), name='weight'),
            nn.LeakyReLU(0.2)
        )
                
        # Linear 
        self.wn_linear = weight_norm(nn.Linear(in_features=filter2, out_features=(num_classes)), name='weight')
        self.apply(initializer)
        
    def forward(self, x):
        # Convolutional Operations
        x = self.wn_conv1(x)
        x = self.wn_conv2(x)
        x = self.wn_conv3(x)
        
        # Linear
        x = x.mean(dim=3).mean(dim=2)
        x = self.wn_linear(x)
        x = F.sigmoid(x)
        
        return x

In [ ]:
# Initialize loss and model
criterion = nn.BCELoss()
model = Model()

# Data Loader
train_loader = get_loader(image_size, batch_size)

# Initialize weights
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(b1, b2))

if torch.cuda.is_available():
    criterion.cuda()
    model = nn.DataParallel(model)
    model.cuda()

In [ ]:
# Training Function
def train(epoch, num_epochs, optimizer, criterion, dataloader, model):
    model.train()
    
    total_loss = 0
    total_acc = 0
    loader_len = len(dataloader)

    for i, data in enumerate(dataloader):
        
        optimizer.zero_grad()
        img, label = data
        if torch.cuda.is_available():
            img = img.cuda()
            label = label.cuda()
        b_size = img.size(0)
    
        # Loss computation
        probs = model(img)
        try:
            loss = criterion(probs, label.float())
        except:
            pdb.set_trace()
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        # Train Accuracy Computation
        _, predicted = torch.max(probs, dim=1)
        correct = torch.sum(torch.eq(predicted, label))
        batch_accuracy = correct.item()/float(b_size)
        total_acc += batch_accuracy
        
        # Print stats
        if i%b_size == b_size-1:
            print("Train [Epoch %d/%d] [Batch %d/%d] [loss: %f, acc: %d%%]" % (epoch, num_epochs, i, 
                                       loader_len, loss.item(), 100 * batch_accuracy))
            
    total_loss = total_loss/float(i+1)
    total_acc = total_acc/float(i+1)
    return total_loss, total_acc

In [ ]:
# Testing Function
def test(epoch, num_epochs, criterion, dataloader, model, mode):
    model.eval()
    
    total_loss = 0
    total_acc = 0
    loader_len = len(dataloader)

    for i, data in enumerate(dataloader):
        
        img, label = data
        if torch.cuda.is_available():
            img = img.cuda()
            label = label.cuda()
        b_size = img.size(0)
    
        # Loss computation
        probs = model(img)
        loss = criterion(probs, label.float())
        total_loss += loss.item()
        
        # Test Accuracy Computation
        _, predicted = torch.max(probs, dim=1)
        correct = torch.sum(torch.eq(predicted, label))
        batch_accuracy = correct.item()/float(b_size)
        total_acc += batch_accuracy
        
        # Print stats
        if i%b_size == b_size-1:
            print("%s [Epoch %d/%d] [Batch %d/%d] [loss: %f, acc: %d%%]" % (mode, epoch, num_epochs, i, 
                                       loader_len, loss.item(), 100 * batch_accuracy))
            
    total_loss = total_loss/float(i+1)
    total_acc = total_acc/float(i+1)
    return total_loss, total_acc

In [ ]:
def save_checkpoint(state, is_best):
    if is_best:
        torch.save(state, 'baseline_chkpt.tar')

In [ ]:
'''
Call Train and Test and save best model
'''
best_valid_acc = 0.0
best_valid_loss = 99999.0

for epoch in range(num_epochs):
    train_loss, train_acc = train(epoch, num_epochs, optimizer, criterion, train_loader, model)
    print('------')
    valid_loss, valid_acc = test(epoch, num_epochs, criterion, valid_loader, model, 'Valid')
    
    print('--------------------------------------------------------------------')
    print("Train ===> [Epoch %d/%d] [Train loss: %f, train acc: %d%%]" % (epoch, num_epochs, 
                                                                          train_loss, 100 * train_acc))
    print("Valid ===> [Epoch %d/%d] [Valid loss: %f, valid acc: %d%%]" % (epoch, num_epochs, 
                                                                          valid_loss, 100 * valid_acc))
    print('--------------------------------------------------------------------')
    
    # Save best model
    is_best = valid_acc >= best_valid_acc
    save_checkpoint({
    'epoch': epoch + 1,
    'state_dict': model.state_dict(),
    'optimizer' : optimizer.state_dict(),
    'best_acc' : valid_acc
    }, is_best)
    